# 0. Librerias

In [97]:
# Genérica
# -----------------------------------------------------------------------------
from time import time
import session_info


# Tratamientos datos
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np


# category encoders
# -----------------------------------------------------------------------------
from category_encoders import OrdinalEncoder


# scikit-learn
# -----------------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# LightGBM
# -----------------------------------------------------------------------------
import lightgbm as lgb


# Gráficos
# -----------------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


session_info.show(html=False)

-----
category_encoders   2.6.1
lightgbm            3.3.5
matplotlib          3.7.2
numpy               1.23.5
pandas              1.5.3
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
-----
IPython             8.14.0
jupyter_client      8.3.0
jupyter_core        5.3.1
notebook            6.5.4
-----
Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Linux-5.15.0-105-generic-x86_64-with-glibc2.31
-----
Session information updated at 2024-05-06 17:36


In [98]:
# Funciones
# -----------------------------------------------------------------------------
def optimize_df_dtypes(df):

    variables_categoricas = [
        'Epoca_Covid',
        'Anio',
        'Trimestre',
        'Mes',
        'Negocio',
        'Cliente',
        'Tipo_Cliente',
        'Distribuidor_MSM',
        'Codigo_Postal',
        'Area_Dist',
        'Zona',
        'ID_CCAA',
        'Comunidad_Autonoma',
        'ID_Provincia',
        'Provincia',
        'Gama',
        'Descripcion_Gama',
        'Formato_1',
        'Formato_2'
        ]

    for i in df.columns:
        if i in variables_categoricas:
            df[i] = df[i].astype("category")
 
    fcols = df.select_dtypes('float').columns
    icols = df.select_dtypes('integer').columns

    df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
    df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')

    return df 

# 1. Cargar datos

In [99]:
# Ruta
path = './data/si-train-3.csv'

# Cargar datos
df = pd.read_csv(path, sep=';')

# Optimizar tipo dato
df = optimize_df_dtypes(df)

# Dataset completo (copia seguridad)
df_complete = df.copy()

# Dataset de validación final
filter_val = ((df.Fecha >= 202402) & (df.Fecha <= 202403)) 
df_val = df[filter_val].reset_index(drop=True)

# Dataset de entrenamiento
filter_train = (df.Fecha <= 202401)
df = df[filter_train].reset_index(drop=True)

df.head()

/tmp/ipykernel_54689/2722970494.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=';')


,Fecha,Epoca_Covid,Anio,Mes,Negocio,Cliente,Tipo_Cliente,Codigo_Postal,Area_Dist,ID_Provincia,...,Egatur,Familitur,IPC,IPRI,Temperatura_Media,Precipitacion,Sell_In_1,Sell_In,STRATIO_CREATED_DATE,exec_month
0,201610,pre-covid,2016,10,UNA,534689,DHC,28005,ZZ,99,...,794.02002,0.0,0.9,100.43,16.59,70.349998,5.95,5.95,2024-05-06T13:05:04.878Z,202405
1,201610,pre-covid,2016,10,UNA,534689,DHC,28005,ZZ,99,...,794.02002,0.0,0.9,100.43,16.59,70.349998,0.00,0.00,2024-05-06T13:05:04.878Z,202405
2,201610,pre-covid,2016,10,UNA,534689,DHC,28005,ZZ,99,...,794.02002,0.0,0.9,100.43,16.59,70.349998,1.08,1.08,2024-05-06T13:05:04.878Z,202405
3,201610,pre-covid,2016,10,UNA,534690,DHC,28045,ZZ,99,...,794.02002,0.0,0.9,100.43,16.59,70.349998,5.95,5.95,2024-05-06T13:05:04.878Z,202405
4,201610,pre-covid,2016,10,UNA,534690,DHC,28045,ZZ,99,...,794.02002,0.0,0.9,100.43,16.59,70.349998,0.96,0.96,2024-05-06T13:05:04.878Z,202405


## 1.1. EDA

In [100]:
# DTypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2787682 entries, 0 to 2787681
Data columns (total 39 columns):
 #   Column                Dtype   
---  ------                -----   
 0   Fecha                 int32   
 1   Epoca_Covid           category
 2   Anio                  category
 3   Mes                   category
 4   Negocio               category
 5   Cliente               category
 6   Tipo_Cliente          category
 7   Codigo_Postal         category
 8   Area_Dist             category
 9   ID_Provincia          category
 10  Provincia             category
 11  ID_CCAA               category
 12  Comunidad_Autonoma    category
 13  Gama                  category
 14  Descripcion_Gama      category
 15  Formato_1             category
 16  Formato_2             category
 17  Num_PdV_CI            float32 
 18  Num_PdV_RU            float32 
 19  Num_PdV_TU            float32 
 20  Impactos_TV           float32 
 21  Sell_In_PP            float64 
 22  Marketing_Nacional

In [101]:
# Separar datasets
dates_complete = df_complete['Fecha'].nunique()
dates_train = df['Fecha'].nunique()
dates_val = df_val['Fecha'].nunique()

print(f"Fechas totales: {dates_complete}")
print(f"Fechas entrenamiento: {dates_train}")
print(f"Fechas validación: {dates_val}")

Fechas totales: 73
Fechas entrenamiento: 70
Fechas validación: 2


In [102]:
sorted(df['Fecha'].unique())

[201610,
 201611,
 201612,
 201701,
 201702,
 201703,
 201704,
 201705,
 201706,
 201707,
 201708,
 201709,
 201710,
 201711,
 201712,
 201801,
 201802,
 201803,
 201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903,
 201904,
 201905,
 201906,
 201907,
 201908,
 201909,
 201910,
 201911,
 201912,
 202001,
 202002,
 202109,
 202110,
 202111,
 202112,
 202201,
 202202,
 202203,
 202204,
 202205,
 202206,
 202207,
 202208,
 202209,
 202210,
 202211,
 202212,
 202301,
 202302,
 202303,
 202304,
 202305,
 202306,
 202307,
 202308,
 202309,
 202310,
 202311,
 202312,
 202401]

In [103]:
# Conteo por Negocio
df['Negocio'].value_counts()

UNE    2136342
UNA     651340
Name: Negocio, dtype: int64

In [104]:
# Conteo por Formato_1
df[['Negocio', 'Formato_1']].value_counts()

Negocio  Formato_1    
UNE      No retornable    1154638
         Retornable        517784
UNA      No retornable     509181
UNE      Barril            463920
UNA      Retornable        142091
         Barril                68
dtype: int64

In [105]:
# Conteo por Negocio-Formato_2
df[['Negocio', 'Formato_2']].value_counts()

Negocio  Formato_2       
UNE      Tercio SR           555273
         Barril              441661
         Tercio              367012
         Lata                361734
UNA      PET 0,50 L          152533
UNE      Quinto              150772
         Cuarto SR           105419
UNA      PET 1,50 L           94536
         PET 0,33 L           90994
         MEDIO LITRO          77118
UNE      Gran Formato         65479
UNA      LITRO                51522
         TERCIO SR            49081
UNE      Medio Litro SR       47225
UNA      PET 5,00 L           36446
         PET 1,00 L           23285
UNE      Duotank              22259
UNA      PET 0,75 L           19807
UNE      Quinto SR            16342
UNA      GRAN FORMATO         13823
         Lata                 11450
         PET GRAN FORMATO      9765
         CR 0,33 L             6454
         CUARTO SR             6183
         CR 0,75 L             6102
UNE      Bag in Box            3166
UNA      11 Litros              807
  

In [106]:
# Eliminar columnas genéricas para todos los experimentos
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Codigo_Postal',
    'Provincia',
    'Comunidad_Autonoma',
    'STRATIO_CREATED_DATE',
    'exec_month'
])

In [107]:
# Contar el número de NaN por columna
num_nan_por_columna = df.isna().sum()
print(num_nan_por_columna)

Epoca_Covid              0
Mes                      0
Negocio                  0
Tipo_Cliente             0
Area_Dist              920
ID_Provincia             0
ID_CCAA                  0
Gama                     0
Descripcion_Gama         0
Formato_1                0
Formato_2                0
Num_PdV_CI               0
Num_PdV_RU               0
Num_PdV_TU               0
Impactos_TV              0
Sell_In_PP               0
Marketing_Nacional       0
Trade_Marketing          0
Marketing_PLV            0
Dispersion_PdV           0
Volumen_Solicitado       0
Tarifa_Eur_Hl            0
Margen_Distribuidor      0
Egatur                   0
Familitur                0
IPC                      0
IPRI                     0
Temperatura_Media        0
Precipitacion            0
Sell_In_1                0
Sell_In                  0
dtype: int64


In [108]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variables independientes: {features}')
print(30*'-')
print(f'Variables categóricas: {categorical_features}')

Variable dependiente: Sell_In
------------------------------
Variables independientes: ['Epoca_Covid', 'Mes', 'Negocio', 'Tipo_Cliente', 'Area_Dist', 'ID_Provincia', 'ID_CCAA', 'Gama', 'Descripcion_Gama', 'Formato_1', 'Formato_2', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Sell_In_PP', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Dispersion_PdV', 'Volumen_Solicitado', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Egatur', 'Familitur', 'IPC', 'IPRI', 'Temperatura_Media', 'Precipitacion', 'Sell_In_1']
------------------------------
Variables categóricas: ['Epoca_Covid', 'Mes', 'Negocio', 'Tipo_Cliente', 'Area_Dist', 'ID_Provincia', 'ID_CCAA', 'Gama', 'Descripcion_Gama', 'Formato_1', 'Formato_2']


## 1.2. Test funcionales

In [109]:
def test_ok():
    print("\033[92m" + 13*"-" + "\033[0m")
    print("\033[92m" + "-- Test OK --" + "\033[0m")
    print("\033[92m" + 13*"-" + "\033[0m")
    
def test_ko():
    print("\033[91m" + 13*"-" + "\033[0m")
    print("\033[91m" + "-- Test KO --" + "\033[0m")
    print("\033[91m" + 13*"-" + "\033[0m")

In [112]:
# Todos los DHC tienen el área ZZ
prueba = (df.Tipo_Cliente == 'DHC') & (df.Area_Dist != 'ZZ')

if df[prueba].shape[0] == 0:
    test_ok()
else:
    test_ko()
    df[prueba].head(1)

-------------
-- Test OK --
-------------


# 2. MLflow

In [ ]:
# Importar MLflow
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec

In [ ]:
# Ver ruta actual desde donde se va a levantar el MLflow server
!pwd

In [ ]:
# Iniciar un servidor de MLflow con alguno de los dos comandos siguientes en una terminal
# $ mlflow server --host 127.0.0.1 --port 5000
# $ mlflow ui

In [ ]:
# Apuntar el notebook al servidor MLflow levantado en la celda anterior
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
# Crear un experimento
# --------------------------------------------------

# Nombre
experiment_name = "MSM_Sell_In_Modelos_Formato"

# Descripción
experiment_description = (
    "Proyecto de comparación de modelos de Sell In en función del Formato 2."
)

# Etiquetas del experimento
experiment_tags = {
    "project_name": "Modelos-Sell-In",
    "problem_type": "Regresión",
    "client": "MSM",
    "mlflow.note.content": experiment_description,
}

# Crear el experimento
produce_sell_in_experiment = mlflow.create_experiment(
    name=experiment_name,
    tags=experiment_tags
)

# Activar el experimento
mlflow.set_experiment(experiment_name)

# 3. Modelos

## 3.1. LightGBM - Modelo Global

El alcance de este experimento es:

**Modelo Global:**
- Negocio: UNE + UNA
- Formatos: Todos

### 3.1.1. Entrenamiento

In [ ]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Modelos_Formato'
run_name = 'Modelo_LightGBM_Experiment_Global'
model_name = 'lightgbm-model-global'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Filtrar por el Formato_1: Global
# ...
# ...

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)



# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 10,
        "num_leaves": 350,
        "min_child_samples": 20,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        ColSpec("string", "Epoca_Covid"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Negocio"),
        ColSpec("string", "Tipo_Cliente"),
        ColSpec("string", "Area_Dist"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        ColSpec("double", "IPC"),
        ColSpec("double", "IPRI"),
        ColSpec("double", "Temperatura_Media"),
        ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/{model_name}'")

In [ ]:
logged_model_lgb_global = f'runs:/{run_id}/{model_name}'
logged_model_lgb_global

### 3.1.2. Predicciones dataset de validación

In [ ]:
# Dataset para comprobar predicciones
df_val.head()

In [ ]:
df_val.info()

In [ ]:
# Castear las columnas de tipo category a object
categorical_columns = df_val.select_dtypes(include=['category']).columns
df_val[categorical_columns] = df_val[categorical_columns].astype('object')

In [ ]:
# Lista de variables dependientes
print(features)

In [ ]:
# Cargar modelo como PyFuncModel.
# logged_model_lgb_global = 'runs:/35f3e23b74f14166a98322282a483129/lightgbm-model-global'
print(logged_model_lgb_global)
loaded_model_lgb = mlflow.pyfunc.load_model(logged_model_lgb_global)

# Prediccion
predictions = pd.Series(loaded_model_lgb.predict(df_val[features]), name='Sell_In_Pred')

In [ ]:
# Columnas del análisis
df_analysis = df_val[['Area_Dist', 'Formato_1', 'Formato_2', 'Sell_In']]

# Concatenar dataset anterior con las predicciones
df_analysis = pd.concat([df_analysis, predictions], axis=1)

df_analysis.head()

In [ ]:
# Agregar el dato por Fecha, Area y Formato
df_analysis = df_analysis.groupby(['Area_Dist', 'Formato_1', 'Formato_2'], as_index=False) \
                         .sum(['Sell_In', 'Sell_In_Pred'])

# Calcular el error en %
df_analysis['Desviacion'] = round((df_analysis.Sell_In_Pred - df_analysis.Sell_In)*100 / df_analysis.Sell_In, 2)

df_analysis

In [ ]:
# Pivot
tabla_desviaciones = df_analysis.pivot_table(index='Area_Dist', columns=['Formato_1', 'Formato_2'], values='Desviacion', aggfunc='mean')

tabla_desviaciones

In [ ]:
# Cambiar el tamaño del mapa de calor
plt.subplots(figsize=(10, 4))  # Ancho x Alto

# Mapa de calor
sns.heatmap(
    tabla_desviaciones,
    annot=True,
    fmt=".2f",
    cmap='RdYlGn',
    vmin=-100,
    vmax=100,
    linewidth=.5
)

# Mostrar el mapa de calor
plt.title('Mapa de Calor de Desviaciones')
plt.xlabel('Formato')
plt.ylabel('Area')

plt.show()